In [19]:
import requests, re, urllib, itertools
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from collections import Counter, Iterable
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from anytree import Node
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk import pos_tag, word_tokenize

def checkAppropriateURL(url):
    parsedURL = urlparse(url)   

    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]

    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    try:
        r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
        if(not r=="text/html"):
            print ("Failed",url,r)
            return False
    except:
        pass
    return True

def scrapeGoogleForAbstracts(query, count):
    words = query.split()
    q = ("+").join(words)
#     print (q)
    query_url_goog = "https://www.google.co.in/search?q=\"" + q + "\""
    data = requests.get(query_url_goog).text
    soup = BeautifulSoup(data, "lxml")
    h3Rows = soup.find_all("h3", {"class":"r"})
    spanRows = soup.find_all("span", {"class":"st"})
    zippedRows = zip(h3Rows,spanRows)
    urls = []
#     print (list(zippedRows))
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL))
    i = 1
    same = 0
    prev = 0
    while(len(urls)<count):
#         print ("Query: ",query)
        tempurl = query_url_goog + "&start=" +str(i*10)
        i+=1
        data = requests.get(tempurl).text
        soup = BeautifulSoup(data, "lxml")
        h3Rows = soup.find_all("h3", {"class":"r"})
        spanRows = soup.find_all("span", {"class":"st"})
        zippedRows = zip(h3Rows,spanRows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]



def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    finalList = [a.strip() for a in list(visible_texts) if a.strip()]
    return "\n".join(finalList)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
#     print (inputText)
    return inputText

def getAbstracts(query, count):
    urls = scrapeGoogleForAbstracts(query, count)
#     print (urls)
#     print ("lenght",len(urls))
    texts = []
    for (query, url) in urls:
#         print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((query,url,text))
    return texts

def getHearstPatterns(word):
    # Pattern, Instances after/before, One/many instances
    patterns = []
    patterns.append((word + " such as", True, True))
    patterns.append(("such " + word + " as", True, True))
    patterns.append(("and other " + word, False, True))
    patterns.append(("or other " + word, False, True))
    patterns.append((word + ", including", True, True))
    patterns.append((word + ", especially", True, True))
    return patterns

finalList = []
for word in ["pizzas", "pizza toppings"]:
    instanceList = []
    queriesList = getHearstPatterns(word)
    for query in queriesList:
        allAbstracts = getAbstracts(query[0], 50)
        for abstractTuple in allAbstracts:
            abstract = abstractTuple[2]
            instanceList.append(abstract)
    finalList.append(instanceList)



Failed https://digitalcommons.liberty.edu/cgi/viewcontent.cgi?article=1016&context=paper_02_03 application/pdf
Failed https://www.tripadvisor.co.za/ShowUserReviews-g261650-d1911240-r539623742-Pasini_s_Cafe-Bicheno_Tasmania.html text/plain
Failed https://www.tripadvisor.ie/ShowUserReviews-g1725258-d2434985-r551248194-Ai_Tre_Archi-City_of_Venice_Veneto.html text/plain
Failed https://biz.dominos.com/documents/1697006/6681236/DOM-227_SmartSliceBrochure_Trifold_8_5x11_Final_LoRes.pdf/2ba35601-2bcc-4313-7ac9-2c909be1838e application/pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [32]:
model = Word2Vec(finalList, min_count=1)

In [37]:
model.wv['toppings']

array([-0.51227206, -1.2847223 ,  3.490544  , -0.93762606,  1.7334481 ,
        0.27087292,  0.66152835, -0.84823054, -0.17715542,  0.07977004,
        0.1507177 , -0.9045556 , -2.8559575 , -1.2382495 , -0.03531567,
       -0.31199107,  1.3665978 , -2.2502768 , -1.5593722 , -0.17395428,
        1.719916  ,  1.2555044 , -2.2604537 ,  0.7508089 , -1.1353773 ,
        2.9665873 ,  0.9214447 , -0.68822056,  0.14849168,  1.8077168 ,
        0.37879112, -1.2270753 , -0.38158682,  0.29182464,  0.03694698,
        2.3279104 , -0.23605627, -0.38577148,  1.5197443 , -0.24042459,
       -0.19521521, -0.5266814 ,  1.5598212 , -0.32768145,  1.0994464 ,
        1.4933176 ,  0.3170104 ,  1.7047279 , -3.6475973 ,  0.86272013,
       -0.03298703,  0.133931  , -1.2155799 ,  1.493747  , -1.050826  ,
       -0.7165058 , -0.51827544,  0.08226824,  1.421184  ,  1.6986766 ,
       -2.9063528 , -0.63867164,  0.00454799, -1.015708  ,  1.4364771 ,
        0.7939562 ,  2.4334238 , -1.9164641 , -1.5604006 ,  0.47